# 60. バンドル調整
## Bundle Adjustment

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] バンドル調整の目的と原理を理解する
- [ ] 再投影誤差の最小化問題を定式化できる
- [ ] ヤコビアン行列とスパース構造を理解する
- [ ] Levenberg-Marquardt法の仕組みを理解する
- [ ] 簡易的なバンドル調整を実装できる
- [ ] Schur補行列によるスパース解法を理解する

---

## 前提知識

- 57: 三角測量と3D復元
- 59: SfM パイプライン基礎
- 最適化理論：最小二乗法、勾配降下法

**難易度**: ★★★★★（上級）  
**推定学習時間**: 120-150分

---

## 1. バンドル調整とは

**バンドル調整（Bundle Adjustment, BA）**は、SfMやSLAMにおいて、カメラ姿勢と3D点を同時に最適化する手法です。

### 名前の由来

「バンドル」は、3D点からカメラへ向かう光線の束（bundle of rays）を指します。これらの光線が画像上で観測点と一致するように調整します。

### なぜバンドル調整が必要か

- 初期復元には累積誤差がある
- 特徴点マッチングにはノイズがある
- 全体的な整合性を取る必要がある

### 最適化の目的関数

$$\min_{\{\mathbf{R}_i, \mathbf{t}_i\}, \{\mathbf{X}_j\}} \sum_{i,j} \rho\left( \|\mathbf{x}_{ij} - \pi(\mathbf{K}_i, \mathbf{R}_i, \mathbf{t}_i, \mathbf{X}_j)\|^2 \right)$$

ここで：
- $\mathbf{x}_{ij}$: カメラ $i$ での点 $j$ の観測
- $\pi$: 投影関数
- $\rho$: ロバストカーネル（Huber, Cauchy など）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Dict, Optional
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix
import time
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

---

## 2. 問題の定式化

### 2.1 パラメータ

最適化するパラメータ：

| パラメータ | 次元 | 説明 |
|-----------|------|------|
| カメラ姿勢 | 6 per camera | 回転（3）+ 並進（3） |
| 3D点 | 3 per point | X, Y, Z |
| 内部パラメータ | 4-5 per camera | f, cx, cy, k1, k2（オプション） |

### 2.2 回転の表現

回転は以下の表現が使われます：

- **回転ベクトル（Rodrigues）**: 3パラメータ、特異点なし
- **クォータニオン**: 4パラメータ、正規化が必要
- **回転行列**: 9パラメータ、直交拘束が必要

バンドル調整では**回転ベクトル**が一般的です。

In [ ]:
def rodrigues_to_rotation_matrix(rvec: np.ndarray) -> np.ndarray:
    """回転ベクトル(Rodrigues)から回転行列へ変換"""
    theta = np.linalg.norm(rvec)
    
    if theta < 1e-10:
        return np.eye(3)
    
    k = rvec / theta  # 単位ベクトル
    K = np.array([
        [0, -k[2], k[1]],
        [k[2], 0, -k[0]],
        [-k[1], k[0], 0]
    ])
    
    R = np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K
    return R

def rotation_matrix_to_rodrigues(R: np.ndarray) -> np.ndarray:
    """回転行列から回転ベクトル(Rodrigues)へ変換"""
    theta = np.arccos(np.clip((np.trace(R) - 1) / 2, -1, 1))
    
    if theta < 1e-10:
        return np.zeros(3)
    
    k = np.array([R[2, 1] - R[1, 2],
                  R[0, 2] - R[2, 0],
                  R[1, 0] - R[0, 1]]) / (2 * np.sin(theta))
    
    return theta * k

# テスト
rvec_test = np.array([0.1, 0.2, 0.3])
R_test = rodrigues_to_rotation_matrix(rvec_test)
rvec_recovered = rotation_matrix_to_rodrigues(R_test)

print("回転ベクトル → 回転行列 → 回転ベクトル の変換テスト")
print(f"元の回転ベクトル: {rvec_test}")
print(f"復元された回転ベクトル: {rvec_recovered}")
print(f"誤差: {np.linalg.norm(rvec_test - rvec_recovered):.6e}")

---

## 3. 合成データの準備

In [ ]:
class BundleAdjustmentProblem:
    """バンドル調整問題のデータ構造"""
    
    def __init__(self, n_cameras: int = 6, n_points: int = 50):
        self.n_cameras = n_cameras
        self.n_points = n_points
        
        # カメラ内部パラメータ（固定）
        self.K = np.array([
            [500, 0, 320],
            [0, 500, 240],
            [0, 0, 1]
        ])
        
        self.image_size = (640, 480)
        
        # 真値の生成
        self._generate_ground_truth()
        
        # 観測データの生成
        self._generate_observations()
    
    def _generate_ground_truth(self):
        """真のカメラ姿勢と3D点を生成"""
        np.random.seed(42)
        
        # 3D点（原点周辺に分布）
        self.points_3d_true = np.random.randn(self.n_points, 3) * 2
        self.points_3d_true[:, 2] = np.abs(self.points_3d_true[:, 2]) + 2
        
        # カメラ姿勢（円周上に配置）
        self.cameras_true = []  # [(rvec, tvec), ...]
        radius = 6
        
        for i in range(self.n_cameras):
            angle = 2 * np.pi * i / self.n_cameras
            
            # カメラ位置
            C = np.array([radius * np.cos(angle), 0, radius * np.sin(angle)])
            
            # 原点を向く回転
            z_axis = -C / np.linalg.norm(C)
            y_axis = np.array([0, 1, 0])
            x_axis = np.cross(y_axis, z_axis)
            x_axis = x_axis / np.linalg.norm(x_axis)
            y_axis = np.cross(z_axis, x_axis)
            
            R = np.vstack([x_axis, y_axis, z_axis])
            t = -R @ C
            
            rvec = rotation_matrix_to_rodrigues(R)
            self.cameras_true.append((rvec, t))
    
    def _generate_observations(self, noise_level: float = 0.5):
        """観測データを生成"""
        self.observations = []  # [(cam_idx, pt_idx, u, v), ...]
        self.visibility = np.zeros((self.n_cameras, self.n_points), dtype=bool)
        
        for cam_idx, (rvec, tvec) in enumerate(self.cameras_true):
            R = rodrigues_to_rotation_matrix(rvec)
            P = self.K @ np.hstack([R, tvec.reshape(-1, 1)])
            
            for pt_idx, X in enumerate(self.points_3d_true):
                X_h = np.append(X, 1)
                x_h = P @ X_h
                
                if x_h[2] <= 0:  # 後方
                    continue
                
                x = x_h[:2] / x_h[2]
                
                # 画像範囲内チェック
                if (0 <= x[0] < self.image_size[0] and 
                    0 <= x[1] < self.image_size[1]):
                    
                    # ノイズを追加
                    x_noisy = x + np.random.randn(2) * noise_level
                    
                    self.observations.append((cam_idx, pt_idx, x_noisy[0], x_noisy[1]))
                    self.visibility[cam_idx, pt_idx] = True
    
    def get_initial_estimate(self, noise_camera: float = 0.1, 
                              noise_point: float = 0.3) -> Tuple[np.ndarray, np.ndarray]:
        """ノイズを加えた初期推定値を取得"""
        # カメラパラメータ
        cameras_init = []
        for rvec, tvec in self.cameras_true:
            rvec_noisy = rvec + np.random.randn(3) * noise_camera
            tvec_noisy = tvec + np.random.randn(3) * noise_camera
            cameras_init.append((rvec_noisy, tvec_noisy))
        
        # 3D点
        points_init = self.points_3d_true + np.random.randn(*self.points_3d_true.shape) * noise_point
        
        return cameras_init, points_init

# 問題の生成
ba_problem = BundleAdjustmentProblem(n_cameras=6, n_points=50)

print(f"カメラ数: {ba_problem.n_cameras}")
print(f"3D点数: {ba_problem.n_points}")
print(f"観測数: {len(ba_problem.observations)}")
print(f"平均観測数/点: {len(ba_problem.observations) / ba_problem.n_points:.1f}")

---

## 4. 再投影誤差関数

### 4.1 投影関数

$$\mathbf{x} = \pi(\mathbf{K}, \mathbf{R}, \mathbf{t}, \mathbf{X}) = \mathbf{K} (\mathbf{R} \mathbf{X} + \mathbf{t})$$

正規化後：

$$\begin{pmatrix} u \\ v \end{pmatrix} = \begin{pmatrix} f_x \frac{X_c}{Z_c} + c_x \\ f_y \frac{Y_c}{Z_c} + c_y \end{pmatrix}$$

In [ ]:
def project_point(K: np.ndarray, rvec: np.ndarray, tvec: np.ndarray, 
                  point_3d: np.ndarray) -> np.ndarray:
    """3D点を画像に投影"""
    R = rodrigues_to_rotation_matrix(rvec)
    point_cam = R @ point_3d + tvec
    
    if point_cam[2] <= 0:
        return np.array([np.inf, np.inf])
    
    point_proj = K @ point_cam
    return point_proj[:2] / point_proj[2]

def compute_residuals(params: np.ndarray, 
                       n_cameras: int, n_points: int,
                       observations: List[Tuple],
                       K: np.ndarray) -> np.ndarray:
    """全観測に対する残差を計算
    
    params: [rvec1, tvec1, rvec2, tvec2, ..., X1, Y1, Z1, X2, Y2, Z2, ...]
    """
    # パラメータの展開
    camera_params = params[:n_cameras * 6].reshape(n_cameras, 6)
    point_params = params[n_cameras * 6:].reshape(n_points, 3)
    
    residuals = []
    
    for cam_idx, pt_idx, u_obs, v_obs in observations:
        rvec = camera_params[cam_idx, :3]
        tvec = camera_params[cam_idx, 3:6]
        point_3d = point_params[pt_idx]
        
        projected = project_point(K, rvec, tvec, point_3d)
        
        residuals.append(projected[0] - u_obs)
        residuals.append(projected[1] - v_obs)
    
    return np.array(residuals)

def compute_total_error(params: np.ndarray, 
                        n_cameras: int, n_points: int,
                        observations: List[Tuple],
                        K: np.ndarray) -> float:
    """総再投影誤差を計算"""
    residuals = compute_residuals(params, n_cameras, n_points, observations, K)
    return np.sqrt(np.mean(residuals ** 2))

print("再投影誤差関数の実装完了")

In [ ]:
# 初期推定値での誤差を計算
cameras_init, points_init = ba_problem.get_initial_estimate(noise_camera=0.05, noise_point=0.2)

# パラメータをフラットな配列に変換
def pack_params(cameras: List[Tuple], points: np.ndarray) -> np.ndarray:
    camera_params = np.array([np.concatenate([rvec, tvec]) for rvec, tvec in cameras]).flatten()
    point_params = points.flatten()
    return np.concatenate([camera_params, point_params])

def unpack_params(params: np.ndarray, n_cameras: int, n_points: int) -> Tuple:
    camera_params = params[:n_cameras * 6].reshape(n_cameras, 6)
    cameras = [(cp[:3], cp[3:6]) for cp in camera_params]
    points = params[n_cameras * 6:].reshape(n_points, 3)
    return cameras, points

# 真値でのパラメータ
params_true = pack_params(ba_problem.cameras_true, ba_problem.points_3d_true)

# 初期推定値でのパラメータ
params_init = pack_params(cameras_init, points_init)

# 誤差の計算
error_true = compute_total_error(params_true, ba_problem.n_cameras, ba_problem.n_points,
                                  ba_problem.observations, ba_problem.K)
error_init = compute_total_error(params_init, ba_problem.n_cameras, ba_problem.n_points,
                                  ba_problem.observations, ba_problem.K)

print(f"真値での再投影誤差 (RMS): {error_true:.4f} pixels")
print(f"初期推定での再投影誤差 (RMS): {error_init:.4f} pixels")

---

## 5. ヤコビアン行列とスパース構造

### 5.1 ヤコビアンの定義

残差 $\mathbf{r}$ のパラメータ $\mathbf{p}$ に関するヤコビアン：

$$\mathbf{J} = \frac{\partial \mathbf{r}}{\partial \mathbf{p}}$$

### 5.2 スパース構造

バンドル調整のヤコビアンは**スパース**です：

- 各観測は1つのカメラと1つの3D点にのみ依存
- 大部分の要素がゼロ

```
J = [ J_c1  0    0   | J_p1  0    0   ]
    [ J_c1  0    0   |  0   J_p2  0   ]
    [  0   J_c2  0   | J_p1  0    0   ]
    [  0   J_c2  0   |  0    0   J_p3 ]
    [ ...                             ]
```

In [ ]:
def compute_jacobian_sparsity(n_cameras: int, n_points: int,
                               observations: List[Tuple]) -> lil_matrix:
    """ヤコビアンのスパースパターンを計算"""
    n_obs = len(observations)
    n_params = n_cameras * 6 + n_points * 3
    
    # 各観測は2つの残差（u, v）を持つ
    sparsity = lil_matrix((n_obs * 2, n_params), dtype=int)
    
    for i, (cam_idx, pt_idx, _, _) in enumerate(observations):
        # カメラパラメータの範囲
        cam_start = cam_idx * 6
        cam_end = cam_start + 6
        
        # 点パラメータの範囲
        pt_start = n_cameras * 6 + pt_idx * 3
        pt_end = pt_start + 3
        
        # 非ゼロ要素をマーク
        sparsity[2*i, cam_start:cam_end] = 1
        sparsity[2*i, pt_start:pt_end] = 1
        sparsity[2*i+1, cam_start:cam_end] = 1
        sparsity[2*i+1, pt_start:pt_end] = 1
    
    return sparsity

# スパースパターンの計算
sparsity = compute_jacobian_sparsity(ba_problem.n_cameras, ba_problem.n_points,
                                      ba_problem.observations)

# 可視化
fig, ax = plt.subplots(figsize=(12, 6))
ax.spy(sparsity.tocsr(), markersize=1)
ax.set_xlabel('Parameters (Cameras | Points)', fontsize=12)
ax.set_ylabel('Observations (residuals)', fontsize=12)
ax.set_title('Jacobian Sparsity Pattern', fontsize=14)

# カメラと点の境界を示す
cam_boundary = ba_problem.n_cameras * 6
ax.axvline(x=cam_boundary, color='red', linestyle='--', linewidth=2, label='Camera|Point boundary')
ax.legend()

plt.tight_layout()
plt.show()

total_elements = sparsity.shape[0] * sparsity.shape[1]
nonzero_elements = sparsity.nnz
sparsity_ratio = 1 - nonzero_elements / total_elements

print(f"ヤコビアンサイズ: {sparsity.shape}")
print(f"非ゼロ要素数: {nonzero_elements}")
print(f"スパース率: {sparsity_ratio * 100:.2f}%")

---

## 6. Levenberg-Marquardt法

### 6.1 ガウス-ニュートン法

非線形最小二乗問題 $\min_\mathbf{p} \|\mathbf{r}(\mathbf{p})\|^2$ に対する反復解法：

$$\mathbf{J}^\top \mathbf{J} \Delta\mathbf{p} = -\mathbf{J}^\top \mathbf{r}$$

$$\mathbf{p}_{new} = \mathbf{p} + \Delta\mathbf{p}$$

### 6.2 Levenberg-Marquardt法

ガウス-ニュートン法に正則化項を追加：

$$(\mathbf{J}^\top \mathbf{J} + \lambda \mathbf{I}) \Delta\mathbf{p} = -\mathbf{J}^\top \mathbf{r}$$

- $\lambda$ が小さい: ガウス-ニュートン法（高速収束）
- $\lambda$ が大きい: 勾配降下法（安定）

$\lambda$ は誤差の減少に応じて適応的に調整。

In [ ]:
def bundle_adjustment_lm(params_init: np.ndarray,
                          n_cameras: int, n_points: int,
                          observations: List[Tuple],
                          K: np.ndarray,
                          max_iterations: int = 50,
                          ftol: float = 1e-6,
                          verbose: bool = True) -> Tuple[np.ndarray, List[float]]:
    """簡易的なLevenberg-Marquardt法によるバンドル調整
    
    Returns:
        params_opt: 最適化されたパラメータ
        errors: 各反復での誤差履歴
    """
    params = params_init.copy()
    lambd = 1e-3  # 初期ダンピング係数
    
    errors = []
    
    for iteration in range(max_iterations):
        # 残差の計算
        residuals = compute_residuals(params, n_cameras, n_points, observations, K)
        error = np.sqrt(np.mean(residuals ** 2))
        errors.append(error)
        
        if verbose and iteration % 10 == 0:
            print(f"Iteration {iteration}: error = {error:.4f}, lambda = {lambd:.2e}")
        
        # ヤコビアンの数値計算
        epsilon = 1e-8
        n_params = len(params)
        n_residuals = len(residuals)
        
        J = np.zeros((n_residuals, n_params))
        
        for j in range(n_params):
            params_plus = params.copy()
            params_plus[j] += epsilon
            residuals_plus = compute_residuals(params_plus, n_cameras, n_points, observations, K)
            J[:, j] = (residuals_plus - residuals) / epsilon
        
        # 正規方程式
        JtJ = J.T @ J
        Jtr = J.T @ residuals
        
        # Levenberg-Marquardt 更新
        for _ in range(10):  # 内部ループ
            try:
                delta = np.linalg.solve(JtJ + lambd * np.eye(n_params), -Jtr)
            except np.linalg.LinAlgError:
                lambd *= 10
                continue
            
            params_new = params + delta
            residuals_new = compute_residuals(params_new, n_cameras, n_points, observations, K)
            error_new = np.sqrt(np.mean(residuals_new ** 2))
            
            if error_new < error:
                params = params_new
                lambd *= 0.5
                break
            else:
                lambd *= 2
        
        # 収束判定
        if len(errors) > 1 and abs(errors[-1] - errors[-2]) < ftol:
            if verbose:
                print(f"Converged at iteration {iteration}")
            break
    
    return params, errors

print("Levenberg-Marquardt 法の実装完了")

In [ ]:
# 小規模な問題で LM 法をテスト
ba_small = BundleAdjustmentProblem(n_cameras=4, n_points=20)
cameras_init_small, points_init_small = ba_small.get_initial_estimate(noise_camera=0.03, noise_point=0.15)
params_init_small = pack_params(cameras_init_small, points_init_small)

print("バンドル調整を実行中...\n")
start_time = time.time()

params_opt_small, errors_small = bundle_adjustment_lm(
    params_init_small, 
    ba_small.n_cameras, 
    ba_small.n_points,
    ba_small.observations, 
    ba_small.K,
    max_iterations=30,
    verbose=True
)

elapsed_time = time.time() - start_time
print(f"\n実行時間: {elapsed_time:.2f} 秒")

In [ ]:
def visualize_optimization_progress(errors: List[float]):
    """最適化の収束を可視化"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 線形スケール
    ax1.plot(errors, 'b-o', markersize=4)
    ax1.set_xlabel('Iteration', fontsize=12)
    ax1.set_ylabel('RMS Reprojection Error (pixels)', fontsize=12)
    ax1.set_title('Convergence (Linear Scale)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # 対数スケール
    ax2.semilogy(errors, 'r-o', markersize=4)
    ax2.set_xlabel('Iteration', fontsize=12)
    ax2.set_ylabel('RMS Reprojection Error (pixels)', fontsize=12)
    ax2.set_title('Convergence (Log Scale)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Bundle Adjustment Optimization Progress', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"初期誤差: {errors[0]:.4f} pixels")
    print(f"最終誤差: {errors[-1]:.4f} pixels")
    print(f"改善率: {(1 - errors[-1] / errors[0]) * 100:.1f}%")

visualize_optimization_progress(errors_small)

---

## 7. SciPy を使った効率的な実装

実際のアプリケーションでは、効率的な最適化ライブラリを使用します。

In [ ]:
def bundle_adjustment_scipy(params_init: np.ndarray,
                             n_cameras: int, n_points: int,
                             observations: List[Tuple],
                             K: np.ndarray) -> Tuple[np.ndarray, dict]:
    """SciPy の least_squares を使ったバンドル調整"""
    
    # スパースパターン
    sparsity = compute_jacobian_sparsity(n_cameras, n_points, observations)
    
    # 最適化の実行
    result = least_squares(
        fun=lambda p: compute_residuals(p, n_cameras, n_points, observations, K),
        x0=params_init,
        jac_sparsity=sparsity,
        method='trf',  # Trust Region Reflective
        verbose=2,
        max_nfev=100
    )
    
    return result.x, result

# より大きな問題で SciPy を使用
print("SciPy によるバンドル調整を実行中...\n")
start_time = time.time()

params_opt_scipy, result_scipy = bundle_adjustment_scipy(
    params_init,
    ba_problem.n_cameras,
    ba_problem.n_points,
    ba_problem.observations,
    ba_problem.K
)

elapsed_time = time.time() - start_time
print(f"\n実行時間: {elapsed_time:.2f} 秒")
print(f"関数評価回数: {result_scipy.nfev}")
print(f"最終コスト: {result_scipy.cost:.4f}")

In [ ]:
# 最適化前後の比較
error_before = compute_total_error(params_init, ba_problem.n_cameras, ba_problem.n_points,
                                    ba_problem.observations, ba_problem.K)
error_after = compute_total_error(params_opt_scipy, ba_problem.n_cameras, ba_problem.n_points,
                                   ba_problem.observations, ba_problem.K)
error_true = compute_total_error(params_true, ba_problem.n_cameras, ba_problem.n_points,
                                  ba_problem.observations, ba_problem.K)

print("=== 再投影誤差の比較 ===")
print(f"真値: {error_true:.4f} pixels")
print(f"最適化前: {error_before:.4f} pixels")
print(f"最適化後: {error_after:.4f} pixels")
print(f"\n改善率: {(1 - error_after / error_before) * 100:.1f}%")

In [ ]:
def visualize_ba_result(ba_problem: BundleAdjustmentProblem,
                         params_init: np.ndarray,
                         params_opt: np.ndarray):
    """バンドル調整の結果を可視化"""
    cameras_init, points_init = unpack_params(params_init, ba_problem.n_cameras, ba_problem.n_points)
    cameras_opt, points_opt = unpack_params(params_opt, ba_problem.n_cameras, ba_problem.n_points)
    
    fig = plt.figure(figsize=(16, 6))
    
    titles = ['Ground Truth', 'Before BA', 'After BA']
    data_list = [
        (ba_problem.cameras_true, ba_problem.points_3d_true),
        (cameras_init, points_init),
        (cameras_opt, points_opt)
    ]
    
    for idx, (title, (cameras, points)) in enumerate(zip(titles, data_list)):
        ax = fig.add_subplot(1, 3, idx + 1, projection='3d')
        
        # 3D点
        ax.scatter(points[:, 0], points[:, 1], points[:, 2],
                   c='blue', s=10, alpha=0.5)
        
        # カメラ
        colors = plt.cm.tab10(np.linspace(0, 1, len(cameras)))
        for i, (rvec, tvec) in enumerate(cameras):
            R = rodrigues_to_rotation_matrix(rvec)
            C = -R.T @ tvec
            ax.scatter(*C, color=colors[i], s=100, marker='o')
            
            direction = R.T @ np.array([0, 0, 1])
            ax.quiver(*C, *direction, color=colors[i], arrow_length_ratio=0.3)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(title)
        ax.view_init(elev=30, azim=45)
    
    plt.suptitle('Bundle Adjustment Results', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_ba_result(ba_problem, params_init, params_opt_scipy)

---

## 8. Schur補行列による効率的な解法

### 8.1 問題の構造

正規方程式：

$$\begin{pmatrix} \mathbf{U} & \mathbf{W} \\ \mathbf{W}^\top & \mathbf{V} \end{pmatrix} \begin{pmatrix} \Delta\mathbf{c} \\ \Delta\mathbf{p} \end{pmatrix} = \begin{pmatrix} \mathbf{e}_c \\ \mathbf{e}_p \end{pmatrix}$$

ここで：
- $\mathbf{U}$: カメラに関するブロック対角行列
- $\mathbf{V}$: 3D点に関するブロック対角行列
- $\mathbf{W}$: カメラ-点の相互作用

### 8.2 Schur補行列

3D点を消去して、カメラのみの縮小系を得る：

$$(\mathbf{U} - \mathbf{W} \mathbf{V}^{-1} \mathbf{W}^\top) \Delta\mathbf{c} = \mathbf{e}_c - \mathbf{W} \mathbf{V}^{-1} \mathbf{e}_p$$

この系は**はるかに小さく**、効率的に解ける。

In [ ]:
def visualize_schur_complement():
    """Schur補行列の概念を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 元の行列構造
    n_cam, n_pt = 6, 20
    size_cam = n_cam * 6
    size_pt = n_pt * 3
    total = size_cam + size_pt
    
    # フルシステム
    ax = axes[0]
    full_matrix = np.zeros((total, total))
    full_matrix[:size_cam, :size_cam] = 1  # U
    full_matrix[size_cam:, size_cam:] = np.eye(size_pt)  # V (block diagonal)
    for i in range(n_pt):
        full_matrix[size_cam + i*3:size_cam + i*3 + 3, 
                    size_cam + i*3:size_cam + i*3 + 3] = 1
    # W (sparse)
    np.random.seed(42)
    for i in range(n_pt):
        cam_idx = np.random.choice(n_cam, 3, replace=False)
        for c in cam_idx:
            full_matrix[c*6:(c+1)*6, size_cam + i*3:size_cam + (i+1)*3] = 0.5
            full_matrix[size_cam + i*3:size_cam + (i+1)*3, c*6:(c+1)*6] = 0.5
    
    ax.imshow(full_matrix, cmap='Blues')
    ax.axhline(y=size_cam-0.5, color='red', linewidth=2)
    ax.axvline(x=size_cam-0.5, color='red', linewidth=2)
    ax.set_title(f'Full System ({total}×{total})', fontsize=12)
    ax.text(size_cam/2, size_cam/2, 'U', fontsize=14, ha='center', va='center', color='white')
    ax.text(size_cam + size_pt/2, size_cam + size_pt/2, 'V', fontsize=14, ha='center', va='center')
    ax.text(size_cam + size_pt/2, size_cam/2, 'W', fontsize=14, ha='center', va='center')
    
    # V^{-1} の計算
    ax = axes[1]
    v_inv = np.zeros((size_pt, size_pt))
    for i in range(n_pt):
        v_inv[i*3:(i+1)*3, i*3:(i+1)*3] = 1
    ax.imshow(v_inv, cmap='Greens')
    ax.set_title(f'V⁻¹ (Block Diagonal, {size_pt}×{size_pt})', fontsize=12)
    
    # Schur補行列
    ax = axes[2]
    schur = np.ones((size_cam, size_cam))
    ax.imshow(schur, cmap='Reds')
    ax.set_title(f'Schur Complement ({size_cam}×{size_cam})', fontsize=12)
    
    plt.suptitle('Schur Complement: Reduction from Full System to Camera-only System', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"元のシステムサイズ: {total}×{total} = {total**2:,} 要素")
    print(f"Schur補行列サイズ: {size_cam}×{size_cam} = {size_cam**2:,} 要素")
    print(f"削減率: {(1 - size_cam**2 / total**2) * 100:.1f}%")

visualize_schur_complement()

---

## 9. ロバストカーネル

### 9.1 外れ値の問題

二乗誤差は外れ値に敏感です。1つの大きな誤差が最適化を支配してしまいます。

### 9.2 ロバストカーネル

| カーネル | 式 | 特徴 |
|----------|-----|------|
| **二乗** | $\rho(r) = r^2$ | 外れ値に敏感 |
| **Huber** | $\rho(r) = \begin{cases} r^2 & |r| \leq \delta \\ 2\delta|r| - \delta^2 & |r| > \delta \end{cases}$ | 小さい誤差は二乗、大きい誤差は線形 |
| **Cauchy** | $\rho(r) = \log(1 + r^2/c^2)$ | 大きい外れ値をさらに抑制 |

In [ ]:
def huber_kernel(residual: float, delta: float = 1.0) -> float:
    """Huber カーネル"""
    if abs(residual) <= delta:
        return residual ** 2
    else:
        return 2 * delta * abs(residual) - delta ** 2

def cauchy_kernel(residual: float, c: float = 1.0) -> float:
    """Cauchy カーネル"""
    return c ** 2 * np.log(1 + (residual / c) ** 2)

def visualize_robust_kernels():
    """ロバストカーネルの可視化"""
    r = np.linspace(-5, 5, 200)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # カーネル関数
    squared = r ** 2
    huber = np.array([huber_kernel(ri, delta=1.0) for ri in r])
    cauchy = np.array([cauchy_kernel(ri, c=1.0) for ri in r])
    
    ax1.plot(r, squared, 'b-', linewidth=2, label='Squared')
    ax1.plot(r, huber, 'r-', linewidth=2, label='Huber (δ=1)')
    ax1.plot(r, cauchy, 'g-', linewidth=2, label='Cauchy (c=1)')
    ax1.set_xlabel('Residual r', fontsize=12)
    ax1.set_ylabel('ρ(r)', fontsize=12)
    ax1.set_title('Robust Kernels', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 15)
    
    # 重み（導関数）
    # w(r) = ρ'(r) / r
    r_nonzero = np.linspace(0.01, 5, 200)
    
    weight_squared = np.ones_like(r_nonzero) * 2
    weight_huber = np.where(r_nonzero <= 1.0, 2, 2 / r_nonzero)
    weight_cauchy = 2 / (1 + r_nonzero ** 2)
    
    ax2.plot(r_nonzero, weight_squared, 'b-', linewidth=2, label='Squared')
    ax2.plot(r_nonzero, weight_huber, 'r-', linewidth=2, label='Huber')
    ax2.plot(r_nonzero, weight_cauchy, 'g-', linewidth=2, label='Cauchy')
    ax2.set_xlabel('|Residual| r', fontsize=12)
    ax2.set_ylabel('Weight', fontsize=12)
    ax2.set_title('Effective Weight for Each Residual', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Robust Kernels for Outlier Handling', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_robust_kernels()

---

## 10. まとめと次のステップ

### 学んだこと

1. **バンドル調整の目的**: カメラ姿勢と3D点を同時に最適化
2. **再投影誤差**: 観測点と投影点の差を最小化
3. **回転の表現**: 回転ベクトル（Rodrigues）が一般的
4. **スパース構造**: ヤコビアンは疎行列
5. **Levenberg-Marquardt法**: ガウス-ニュートン + 正則化
6. **Schur補行列**: 効率的なスパース解法
7. **ロバストカーネル**: 外れ値への対処

### 重要な数式

| 概念 | 数式 |
|------|------|
| 目的関数 | $\min \sum_{i,j} \|\mathbf{x}_{ij} - \pi(\mathbf{P}_i, \mathbf{X}_j)\|^2$ |
| ガウス-ニュートン | $\mathbf{J}^\top \mathbf{J} \Delta\mathbf{p} = -\mathbf{J}^\top \mathbf{r}$ |
| Levenberg-Marquardt | $(\mathbf{J}^\top \mathbf{J} + \lambda \mathbf{I}) \Delta\mathbf{p} = -\mathbf{J}^\top \mathbf{r}$ |

### 実用的なライブラリ

- **Ceres Solver** (C++): Google 製、高性能
- **g2o** (C++): グラフ最適化
- **GTSAM** (C++): 因子グラフベース
- **scipy.optimize.least_squares** (Python): 手軽に使える

### 次のノートブック

**61. Ray Casting と座標系**では：
- カメラからのレイの生成
- ピクセル座標からワールド座標への変換
- NeRF の基礎となる概念

---

## 11. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. バンドル調整の名前の由来は何ですか？

2. バンドル調整で最適化されるパラメータは何ですか？

3. なぜヤコビアン行列がスパースになるのですか？

4. Levenberg-Marquardt法のダンピング係数 λ の役割は？

5. Schur補行列を使う利点は何ですか？

6. ロバストカーネルが必要な理由は？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. 名前の由来:
       - "Bundle" は光線の束を意味する
       - 3D点からカメラへ向かう光線の束が、画像上の観測点と
         一致するように調整することから
    
    2. 最適化されるパラメータ:
       - カメラ姿勢: 回転（3パラメータ）+ 並進（3パラメータ）
       - 3D点の座標: X, Y, Z（3パラメータ）
       - オプション: カメラ内部パラメータ（f, cx, cy, 歪み係数）
    
    3. スパース性の理由:
       - 各観測は1つのカメラと1つの3D点にのみ依存
       - 残差のヤコビアンは、その観測に関係するパラメータ以外はゼロ
       - カメラ数 × 点数 の組み合わせの大部分が見えない関係
    
    4. ダンピング係数 λ の役割:
       - λ 小: ガウス-ニュートン法に近づく（高速収束、不安定）
       - λ 大: 勾配降下法に近づく（安定だが遅い）
       - 誤差が減少すれば λ を減らし、増加すれば λ を増やす
    
    5. Schur補行列の利点:
       - 3D点のパラメータを消去し、カメラのみの系に縮小
       - 3D点は通常カメラより多いため、大幅な計算量削減
       - 縮小系を解いた後、3D点は容易にバックサブスティチューション
    
    6. ロバストカーネルが必要な理由:
       - 二乗誤差は外れ値（誤マッチング、動く物体）に敏感
       - 1つの大きな誤差が最適化全体を歪める
       - Huber, Cauchy などのカーネルで外れ値の影響を抑制
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [59. SfM パイプライン基礎](59_sfm_pipeline_basics_v1.ipynb)
- **次のノートブック**: [61. Ray Casting と座標系](61_ray_casting_coordinates_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)